## Sentinel-1 data download using sentinelsat

In [2]:
# import required packages
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

import os

Definition of the environment variables and settings for downloads 

In [3]:
## change directory for download
os.chdir('D:\eo_data')

## Define search parameters

# get geoJSON file with region extent
pathJSONFile="//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/JSON_Ibague/IbagueJSON.geojson"

# set search dates
start_date = '20170401'
end_date = '20170405'

## Specify ESA scihub credentials

# store in variables
scihub_user = 'asalazarr'
scihub_pass = 'tila8sude'


Making the call to the API

In [4]:
# connect to the API
api = SentinelAPI(scihub_user, scihub_pass, 'https://scihub.copernicus.eu/dhus')

# download single scene by known product id
# api.download(<product_id>)

# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(read_geojson(pathJSONFile))
products = api.query(footprint,
                     date = (start_date, end_date),
                     platformname = 'Sentinel-1')

# download all results from the search
api.download_all(products)

SentinelAPIError: HTTP status 503 Service Unavailable: 
The Copernicus Open Access Hub

# The Copernicus Open Access Hub will be back soon!

Sorry for the inconvenience,  
we're performing some maintenance at the moment.  

<https://scihub.copernicus.eu/news/News00296>

We'll be back online shortly!

In [4]:
# Get the metadata of the downloaded files
down_files = api.download_all(products)[0]

In [37]:
# Unzip files
import zipfile
import os

for im_id in down_files.keys():
    file_name = down_files.get(im_id).get('title')
    print('Unzipping ' + file_name)
    if not os.path.exists('Ibague'):
        os.makedirs('Ibague')
        print 'Ibague' + ' was created'
    zip_ref = zipfile.ZipFile(file_name+'.zip', 'r')
    zip_ref.extractall('Ibague')
    zip_ref.close()


Unzipping S1B_IW_RAW__0SSV_20170403T104202_20170403T104235_004995_008BC6_71EF


## Pre-processing using SNAP


In [5]:
down_files.keys()

[u'515c867e-24ad-4b62-87ea-fab40eac376b',
 u'4f324c66-865a-46a8-a35e-ccb3da946980',
 u'd6d804e3-b03f-434b-8ca6-fd2f66ba6e43']

In [2]:
import snappy

from snappy import ProductIO
from snappy import HashMap
import shutil
import os  
import ast

from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [3]:

## read Sentinel-1 L1 SLC product
product = ProductIO.readProduct("C:/Users/asalazar/Documents/Ibague/S1B_IW_GRDH_1SSV_20170403T104206_20170403T104231_004995_008BC6_228A.SAFE/manifest.safe")


In [4]:
print(type(product))

# create a HashMap for GPF parameters
apply_orbit = HashMap()
# define the name of the output
name = product.getName()[:32]
orbit_output = "C:/Users/asalazar/Documents/Ibague/" + name + "_orbit_"

# create the orbit product
product = GPF.createProduct("Apply-Orbit-File", apply_orbit, product)
#product.dispose()

# write the results
# ProductIO.writeProduct(orbit_pro, orbit_output, 'BEAM-DIMAP')
# print orbit_output + ' was written'

<type 'org.esa.snap.core.datamodel.Product'>


The next pre-processing steps need to be done for each polarization

In [5]:
pols = ['VV', 'VH']
list(product.getBandNames())

['Amplitude_VV', 'Intensity_VV']

In [6]:

# define the calibration parameters
calibration = HashMap()
calibration.put('outputSigmaBand', True)
calibration.put('sourceBands', 'Intensity_' + 'VV')
calibration.put('selectedPolarisations', 'VV')
calibration.put('outputImageScaleInDb', False)

name = product.getName()[:32]
#calib = "C:/Users/asalazar/Documents/Ibague/" + name + "_calibrate_" + 'VV'

product = GPF.createProduct("Calibration", calibration, product)
#orbit_pro.dispose()

#ProductIO.writeProduct(calib_pro, calib, 'BEAM-DIMAP')
#print 'the calibration product was writed'
#calibrates.append(calib_pro)

### Speckle filtering 

How is it different time-series speckle filtering?

In [8]:

# define the speckle filtering parameters
specklefilter = HashMap()
specklefilter.put('outputSigmaBand', True)
specklefilter.put('sourceBandNames', 'Sigma0_' + 'VV')
specklefilter.put('filter', 'Gamma Map')

product = GPF.createProduct("Speckle-Filter", specklefilter, product)
#calib_pro.dispose()

#name = calib_pro.getName()[:32]
#speckle_output = speckle_path + name + "_spk_" + polarization
#ProductIO.writeProduct(speckle_pro, speckle_output, 'BEAM-DIMAP')
#filters.append(speckle_pro)

### Terrain correction

In [9]:

# define terrain correction parameters
terrain_correction = HashMap()
terrain_correction.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
terrain_correction.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
terrain_correction.put('applyRadiometricNormalization', True)
terrain_correction.put('demName', 'SRTM 3Sec')
terrain_correction.put('pixelSpacingInMeter', 10.0)
terrain_correction.put('sourceBands', 'Sigma0_' + 'VV')
terrain_correction.put('mapProjection', 'WGS84(DD)')

#terrain_correction.put('saveSigmaNought', True)
#terrain_correction.put('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')

product = GPF.createProduct("Terrain-Correction", terrain_correction, product)
#speckle_pro.dispose()

#terrain_output = terrain_path + name + "_corrected_" + polarization
#ProductIO.writeProduct(terrain_pro, terrain_output, 'BEAM-DIMAP')

### Subset area of interest

In [15]:
#def SubsetSentProduct(products,polarization,wkt,Subset_path,write_Product):

# read the polygon to make the subset
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
geom = geojson_to_wkt(read_geojson(pathJSONFile))

# subset operation parameters
subsettings = HashMap()
subsettings.put('geoRegion', geom)
subsettings.put('outputImageScaleInDb', False)
subsettings.put('sourceBandNames', 'Sigma0_' + 'VV')

product = GPF.createProduct("Subset", subsettings, product)
#terrain_pro.dispose()


#name = product.getName()[:32]
#ProductIO.writeProduct(SubsetProduct, Subset_output, 'BEAM-DIMAP')
#print 'the Subset product was writed'
#subsets.append(SubsetProduct)
#return(subsets)

### Logaritmic transformation (to dB)

In [16]:

# set log-scale transformation parameters
paramToDB = HashMap()
paramToDB.put('sourceBandNames', 'Sigma0_' + 'VV')

# make the log transformation
product = GPF.createProduct("LinearToFromdB", paramToDB, product)
#SubsetProduct.dispose()

# write the results
#dB_output = "C:/Users/asalazar/Documents/Ibague/_dB_"
#ProductIO.writeProduct(targetDB, dB_output, 'BEAM-DIMAP')


In [17]:
list(product.getBandNames())

['Sigma0_VV_db']

### Texture analysis

In [18]:

# paramaters for GLCM texture analysis
paramGLCM = HashMap()
paramGLCM.put('sourceBandNames', 'Sigma0_' + 'VV')
paramGLCM.put('windowSizeStr', '5x5')
paramGLCM.put('quantizerStr', 'Probabilistic Quantizer')
paramGLCM.put('quantizationLevelsStr', '16')
paramGLCM.put('displacement','4' )
paramGLCM.put('outputContrast','true')
paramGLCM.put('outputDissimilarity','true')
paramGLCM.put('outputHomogeneity','true')
paramGLCM.put('outputASM','true')
paramGLCM.put('outputEnergy','true')
paramGLCM.put('outputMean','true')
paramGLCM.put('outputVariance','true')
paramGLCM.put('outputCorrelation','true')

product = GPF.createProduct("GLCM", paramGLCM, product)

#GLCM_output = "C:/Users/asalazar/Documents/Ibague/_subset_VV"

#ProductIO.writeProduct(targetGCLM, GLCM_output, 'BEAM-DIMAP')


In [19]:
list(product.getBandNames())

['Sigma0_VV_db_Contrast',
 'Sigma0_VV_db_Dissimilarity',
 'Sigma0_VV_db_Homogeneity',
 'Sigma0_VV_db_ASM',
 'Sigma0_VV_db_Energy',
 'Sigma0_VV_db_MAX',
 'Sigma0_VV_db_Entropy',
 'Sigma0_VV_db_GLCMMean',
 'Sigma0_VV_db_GLCMVariance',
 'Sigma0_VV_db_GLCMCorrelation']

In [21]:
from snappy import jpy

def createProgressMonitor():
    PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem = jpy.get_type('java.lang.System')
    monitor = PWPM(JavaSystem.out)
    return monitor

ProductIO.writeProduct(product, 'C:/Users/asalazar/Documents/Ibague/GLCM_output', 'BEAM-DIMAP', pm = createProgressMonitor())